In [1]:
import numpy as np

states = 3
actions = 2

<div>
<img src="figure.png" width="500"/>
</div>

## 1 a

In [2]:
policy = np.full((states, actions), 0.5)
print(policy)

[[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]]


## 1b

In [3]:
value = np.empty((), dtype=object)
value[()] = (0, 0, 1, 1/actions)
opinion = np.full((states, actions), value, dtype = "f, f, f, f")
print(opinion)

[[(0., 0., 1., 0.5) (0., 0., 1., 0.5)]
 [(0., 0., 1., 0.5) (0., 0., 1., 0.5)]
 [(0., 0., 1., 0.5) (0., 0., 1., 0.5)]]


## 2a

In [4]:
def prob_to_certanity(prob_matrix):
    rows = prob_matrix.shape[0]
    cols = prob_matrix.shape[1]
    cert_matrix = np.zeros((rows, cols), dtype = "f, f, f, f")
    for state in range(rows):
        for action in range(cols):
            p = prob_matrix[state, action]
            cert_matrix[state, action] = (p, 1-p, 0, p)
    return(cert_matrix)

In [5]:
policy_cert = prob_to_certanity(policy)
print(policy_cert)

[[(0.5, 0.5, 0., 0.5) (0.5, 0.5, 0., 0.5)]
 [(0.5, 0.5, 0., 0.5) (0.5, 0.5, 0., 0.5)]
 [(0.5, 0.5, 0., 0.5) (0.5, 0.5, 0., 0.5)]]


## 2b

In [6]:
def beliefConstraintFusion(matrix1, matrix2):
    rows = matrix1.shape[0]
    cols = matrix2.shape[1]
    fused = np.zeros((rows, cols), dtype = "f, f, f, f")
    for state in range(rows):
        for action in range(cols):
            b1 = (matrix1[state, action])[0]
            d1 = (matrix1[state, action])[1]
            u1 =  (matrix1[state, action])[2]
            a1 = (matrix1[state, action])[3]

            b2 = (matrix2[state, action])[0]
            d2 = (matrix2[state, action])[1]
            u2 =  (matrix2[state, action])[2]
            a2 = (matrix2[state, action])[3]

            harmony = b1*b2 + b1*u2 + b2*u1
            conflict = b1*d2 + b2*d1
            b = harmony / (1 - conflict)
            u = u1 * u2 / (1 - conflict)
            a = (a1 * (1 - u1) + a2 * (1 - u2)) / (2 - u1 - u2)
            d = 1 - (b + u)

            (fused[state, action])[0] = b
            (fused[state, action])[1] = d
            (fused[state, action])[2] = u
            (fused[state, action])[3] = a
    return fused

In [7]:
fused = beliefConstraintFusion(policy_cert, opinion)
print(fused)

[[(0.5, 0.5, 0., 0.5) (0.5, 0.5, 0., 0.5)]
 [(0.5, 0.5, 0., 0.5) (0.5, 0.5, 0., 0.5)]
 [(0.5, 0.5, 0., 0.5) (0.5, 0.5, 0., 0.5)]]


## 3

In [8]:
def cert_to_prob(cert_matrix):
    rows = cert_matrix.shape[0]
    cols = cert_matrix.shape[1]
    prob_matrix = np.zeros((rows, cols))
    for state in range(rows):
        for action in range(cols):
            b = (cert_matrix[state, action])[0]
            u = (cert_matrix[state, action])[2]
            a = (cert_matrix[state, action])[3]
            prob_matrix[state, action] = b + a * u
    return prob_matrix

In [9]:
policy = cert_to_prob(fused)
print(policy)

[[0.5 0.5]
 [0.5 0.5]
 [0.5 0.5]]
